In [111]:
import matplotlib.pyplot as plt
%matplotlib inline

In [112]:
benchmarks = ['gcc', 'lbm', 'mcf', 'namd', 'xalancbmk']
complete_result = {}

In [154]:
def extract_counts(file_path):
    counters = {
        'mem_inst_retired.any': [], 
        'branch-instructions': [],
        'branch-misses': [], 
        'L1-icache-load-misses': [], 
        'L1-dcache-loads': [], 
        'L1-dcache-load-misses': [], 
        'L2-loads': [],
        'L2-load-misses': [], 
        'LLC-loads': [], 
        'LLC-load-misses': [], 
        'cycles': [], 
        'instructions': [],
    }
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            value, key = line.split()[1:3]
            counters[key].append(float(value.replace(',', '')))
            
    # Sanity Check
    lengths = [len(counters[key]) for key in counters]
    for i in range(1, len(lengths)):
        assert lengths[i] == lengths[i-1]
    print(f'Number of Count samples: {lengths[0]}')
    return counters

def extract_power(file_path):
    counters = {
        'joule': [], 
        'time': []
    }
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            time, joule = line.split()[0:2]
            counters['joule'].append(float(joule))
            counters['time'].append(float(time))
    
    # Sanity Check
    lengths = [len(counters[key]) for key in counters]
    for i in range(1, len(lengths)):
        assert lengths[i] == lengths[i-1]
    print(f'Number of Power samples: {lengths[0]}')
    return counters

def extract_results(counters):
    results = {
        'IPC': [],
        'branch-pred-acc': [],
        'L1-cahce-acc': [],
        'L2-cache-acc': [],
        'L3-cache-acc': [],
        'L1-hit-rate': [],
        'L2-hit-rate': [],
        'L3-hit-rate': [],
        'memory-reqs': [],
        'power (watt)': [],
    }
    normal_len = len(counters['instructions'])
    for i in range(normal_len):
        results['IPC'].append(counters['instructions'][i] / counters['cycles'][i])
        results['branch-pred-acc'].append(1 - counters['branch-misses'][i] / counters['branch-instructions'][i])
        results['L1-cahce-acc'].append(counters['L1-dcache-loads'][i])
        results['L2-cache-acc'].append(counters['L2-loads'][i])
        results['L3-cache-acc'].append(counters['LLC-loads'][i])
        results['L1-hit-rate'].append(1 - counters['L1-dcache-load-misses'][i] / counters['L1-dcache-loads'][i])
        results['L2-hit-rate'].append(1 - counters['L2-load-misses'][i] / counters['L2-loads'][i])
        results['L3-hit-rate'].append(1 - counters['LLC-load-misses'][i] / counters['LLC-loads'][i])
        L1_hits = counters['L1-dcache-loads'][i] - counters['L1-dcache-load-misses'][i]
        L2_hits = counters['L2-loads'][i] - counters['L2-load-misses'][i]
        L3_hits = counters['LLC-loads'][i] - counters['LLC-load-misses'][i]
        results['memory-reqs'].append(counters['mem_inst_retired.any'][i] - L1_hits - L2_hits - L3_hits)
    
    power_len = len(counters['joule'])
    for i in range(power_len-2):
        results['power (watt)'].append(counters['joule'][i] / (counters['time'][i+1] - counters['time'][i]))
        
    return results

def smooth(arr, window):
    smoothed = []
    for i in range(len(arr)):
        left = int(max(0, i - window // 2))
        right = int(min(len(arr), i + window // 2 + 1))
        window_values = arr[left:right]
        smoothed_value = sum(window_values) / len(window_values)
        smoothed.append(smoothed_value)
    return smoothed


In [155]:
for benchmark in benchmarks:
    print("--------------------------")
    print(f"Processing {benchmark}......")
    counters = extract_counts(f'./math/{benchmark}-math.count')
    power_counters = extract_power(f'./power/{benchmark}-phy.count')
    counters.update(power_counters)
    complete_result[benchmark] = extract_results(counters)

--------------------------
Processing gcc......
Number of Count samples: 274
Number of Power samples: 280
--------------------------
Processing lbm......
Number of Count samples: 526
Number of Power samples: 528
--------------------------
Processing mcf......
Number of Count samples: 238
Number of Power samples: 240
--------------------------
Processing namd......
Number of Count samples: 476
Number of Power samples: 478
--------------------------
Processing xalancbmk......
Number of Count samples: 1377
Number of Power samples: 1351


In [156]:
def plot_and_save_graphs(metric, smooth_ratio):
    plt.figure(figsize=(15, 10))
    for benchmark, result in complete_result.items():
        n = len(result[metric])
        window = n*smooth_ratio
        plt.plot([50*i for i in range(n)], smooth(result[metric], window), label=benchmark)
    plt.legend()
    plt.title(f"Plot of {metric} for Different Benchmarks", fontsize=20)
    plt.xlabel("Time in (ms)", fontsize=15)
    plt.ylabel(f"{metric}", fontsize=15)
    plt.tight_layout()
    plt.savefig(f"plots/{metric}.png")
    plt.close()

In [157]:
for key in complete_result['gcc'].keys():
    plot_and_save_graphs(metric=key, smooth_ratio=0.1)